# Create a large cutout of coadded images

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px" alt="Rubin Observatory logo, a graphical representation of turning stars into data.">
<br>
Contact authors: Christina Williams, Melissa Graham <br>
Last verified to run: 2024-05-13 <br>
LSST Science Pipelines version: Weekly 2024_16 <br>
Container Size: Large <br>
Targeted learning level: Intermediate <br>

**Description:** Create a large custom `deepCoadd` cutout.

**Skills:** Identify tracts and patches and combine them into a large custom `deepCoadd` cutout.

**LSST Data Products:** deepCoadd_calexp

**Packages:** lsst.ip.diffim.GetTemplateTask

**Credit:** Thanks to Aline Chu for raising in the Rubin Community Forum the issue of how to create patch- and tract-spanning cutouts. Thanks to Gregory Dubois-Felsmann and Lauren MacArthur who helped in the Forum.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0.lsst.io">dp0.lsst.io</a>.
Questions are welcome as new topics in the 
<a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> 
of the Rubin Community Forum. 
Rubin staff will respond to all questions posted there.

## 1. Introduction

At the moment, the image cutout service demonstrated in DP0.2 tutorial notebook
13a must be passed a tract and patch, and can only create a cutout from a single
patch.

This will always be a contraint for single processed visit images (PVIs,
also known as `calexp`s).
In the future, users will be able to request `deepCoadd` cutouts from the 
Rubin image cutout service which span patch and tract boundaries.

In the meantime, this tutorial demonstrates how to use the `GetTemplateTask`
to create a single cutout image with contributions from multiple adjacent
patches and tracts. 

In the LSST Science Pipelines, `GetTemplateTask` is used to create a template
image from `deepCoadd` images for a given PVI, in order to perform Difference Image Analysis.

As described in the documentation for
<a href="https://pipelines.lsst.io/modules/lsst.ip.diffim/tasks/lsst.ip.diffim.GetTemplateTask.html">GetTemplateTask</a>
and its
<a href="https://pipelines.lsst.io/py-api/lsst.ip.diffim.GetTemplateTask.html#lsst.ip.diffim.GetTemplateTask.run">run method</a>,
the function of this task is to:

> *Build a template from existing coadds, which may span multiple tracts. The assembled template inherits the WCS of the selected skymap tract and the resolution of the template exposures. Overlapping box regions of the input template patches are pixel by pixel copied into the assembled template image. There is no warping or pixel resampling. Pixels with no overlap of any available input patches are set to nan value and NO_DATA flagged.*

>*Where the tracts overlap, the resulting template image is averaged. The PSF on the template is created by combining the CoaddPsf on each template image into a meta-CoaddPsf.*

**Related tutorials:**
This tutorial assumes some familiarity with the following concepts.
DP0.2 tutorial notebooks that demonstrate these concepts more fully are listed in parentheses.
It is not necessary to run through all of these other tutorials before doing this one,
but they might be useful as reference.

 * the butler (04a, 04b)
 * the TAP service (02)
 * image display (03a, 03b)

### 1.1. Import packages

Import packages from the LSST Science Pipelines (top) and 
general python packages (bottom).

In [ ]:
import lsst.afw.display as afwDisplay
import lsst.geom as geom
from lsst.rsp import get_tap_service, retrieve_query
from lsst.daf.butler import Butler, CollectionType
from lsst.ip.diffim import GetTemplateTask
from lsst.afw.image import Image
from lsst.afw.geom import makeSkyWcs

import os, sys # delete sys later
import matplotlib.pyplot as plt
import numpy as np
from astropy.wcs import WCS

# Added CCW to sanity check WCS
from photutils.aperture import SkyCircularAperture
from astropy import units as u
from astropy.coordinates import SkyCoord


### 1.2. Define functions and parameters

Set the `afwDisplay` backend to `matplotlib`.

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')

Instantiate the TAP service.

In [ ]:
service = get_tap_service("tap")

Instantiate the butler.

In [ ]:
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler(config, collections=collection)

## 2. Prepare to create the cutout

### 2.1. Define central coordinates and box size

Define the central coordinates and the size of the side of the desired cutout.

For this example, the coordinates 50.5265, -39.7589 are known to be near the intersection of patches and tracts.

The size of the cutout will be constrained by the container size selected
when the Notebook Aspect session was started, because all the images
that contribute to the cutout must be held in memory simultaneously
and passed to `GetTemplateTask` (this will become clear in Section 3).

In [ ]:
my_ra = 50.5265
my_dec = -39.7589
box_side_deg = 0.2

Define the extent of the cutout in RA and Dec. Include the `cos(dec)` factor.

In [ ]:
ra1 = my_ra + 0.5 * box_side_deg / np.cos(np.radians(my_dec))
ra2 = my_ra - 0.5 * box_side_deg / np.cos(np.radians(my_dec))
dec1 = my_dec - 0.5 * box_side_deg
dec2 = my_dec + 0.5 * box_side_deg
print('ra1 = %8.4f,  ra2 = %8.4f,  delta RA = %6.3f' %
      (ra1, ra2, np.abs(ra2-ra1)))
print('dec1 = %8.4f, dec2 = %8.4f, delta Dec = %6.3f' %
      (dec1, dec2, np.abs(dec2-dec1)))

Define corners of the cutout.

The order is: SE, SW, NW, NE, as drawn in the plot above.

In [ ]:
corners = np.asarray([[ra1, dec1], [ra2, dec1],
                      [ra2, dec2], [ra1, dec2]],
                     dtype='float')

Option to create a little plot to visualize the corners.

Note that the figure is forced to 3:2 dimensions size.

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.plot(my_ra, my_dec, '*', ms=10, color='blue')
clabels = ['1. SE', '2. SW', '3. NW', '4. NE']
for corner, clabel in zip(corners, clabels):
    plt.text(corner[0], corner[1], clabel, ha='center', va='center')
plt.gca().invert_xaxis()
plt.xlim([ra1+0.05, ra2-0.05])
plt.ylim([dec1-0.05, dec2+0.05])
plt.xlabel('RA')
plt.ylabel('Dec')
plt.show()

### 2.2. Identify nearest patch to use as anchor image

Use the butler to find the tract and patch within which
the central coordinates of the desired cutouts are most centered.

This will be the tract and patch used as the "anchor image"
when creating the big cutout.

In [ ]:
my_point = geom.SpherePoint(my_ra * geom.degrees,
                            my_dec * geom.degrees)
skymap = butler.get('skyMap')
tract = skymap.findTract(my_point)
patch = tract.findPatch(my_point)
anchor_tract = tract.tract_id
anchor_patch = patch.getSequentialIndex()
print('anchor tract and patch: ', anchor_tract, anchor_patch)
del my_point, skymap, tract, patch

### 2.3. Identify additional tracts and patches

In the future, it will be possible to write ADQL statements
that use the INTERSECTS functionality to identify all patches
(or images) that overlap with a defined polygon.

For now, use the TAP service to search the `coaddPatches` table
for images with central coordinates that indicate the patch
could *potentially* overlap with the defined extent of the 
desired cutout.

Increase the extent of the search by the patch size to make sure
no overlapping patch is missed.
This may well end up identifying a few patches that do not, in the end,
overlap with the extent of the desired cutout -- and that's ok.

In [ ]:
sra1 = ra1 + 0.22
sra2 = ra2 - 0.22
sdec1 = dec1 - 0.22
sdec2 = dec2 + 0.22

scorners = np.asarray([[sra1, sdec1], [sra2, sdec1],
                       [sra2, sdec2], [sra1, sdec2]],
                      dtype='float')

Create the TAP query.

In [ ]:
string_polygon = ""
for c, scorner in enumerate(scorners):
    s1 = str(np.round(scorner[0], 4))
    s2 = str(np.round(scorner[1], 4))
    string_polygon += s1 + ', ' + s2
    if c < 3:
        string_polygon += ', '
    del s1, s2

query = "SELECT lsst_patch,lsst_tract,s_dec,s_ra "\
        "FROM dp02_dc2_catalogs.CoaddPatches "\
        "WHERE CONTAINS(POINT('ICRS', s_ra, s_dec), "\
        "POLYGON('ICRS', "+string_polygon+"))=1"
print(query)

Run the TAP query.

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

If the returned job phase was "error", uncomment the following to review the error.

In [ ]:
# job.raise_if_error()

Retrieve the TAP results.

In [ ]:
results = job.fetch_result().to_table()

Option to show the results.

In [ ]:
# results

Hold the tract and patch values for images that 
will contribute, in addition to the anchor image,
to the final custom cutout in `add_tracts_and_patches`.

In [ ]:
temp = []
for result in results:
    tract = result['lsst_tract']
    patch = result['lsst_patch']
    if (tract != anchor_tract) | (patch != anchor_patch):
        temp.append([tract, patch])
add_tracts_and_patches = np.asarray(temp, dtype='int')
del temp

print('add these to the anchor patch: ')
print(add_tracts_and_patches)

## 3. Create the custom cutout

Define the filter of the desired custom cutout.

In [ ]:
use_filter = 'r'

Define the `dataId` for the anchor image.

Use the butler to retrieve the anchor image and its World Coordinate System (WCS).

In [ ]:
anchor_dataId = {'tract': anchor_tract, 'patch': anchor_patch,
                 'band': use_filter}
anchor_image = butler.get('deepCoadd_calexp', dataId=anchor_dataId)
anchor_wcs = butler.get('deepCoadd_calexp.wcs', dataId=anchor_dataId)

Visually inspect the `anchor_image`. The below demonstration plots a circle centered at `my_ra` and `my_dec`, with diameter equal to the desired image box edge outlined above. The map indicates that the desired location is near the patch and tract edge and joining to other deepCoadds is required in order to display a contiguous image. 

In [ ]:
radius = box_side_deg/2. * u.deg
coord = SkyCoord(ra=my_ra*u.degree, dec=my_dec*u.degree, frame='icrs')

wcs = WCS(anchor_image.getWcs().getFitsMetadata())
plt.subplot(projection=WCS(anchor_image.getWcs().getFitsMetadata()))
extent = (anchor_image.getBBox().beginX, anchor_image.getBBox().endX,
                 anchor_image.getBBox().beginY, anchor_image.getBBox().endY)

plt.imshow(anchor_image.image.array,vmin=.1,vmax=1,extent=extent, origin='lower',cmap='gray')

aperture = SkyCircularAperture(coord, r=radius)
pix_aperture = aperture.to_pixel(wcs)
pix_aperture.plot(color='r',lw=3)


Create a bounding box for the new image using the anchor image's WCS.

In [ ]:
# this should be deleted:

corner_SE_deg = geom.SpherePoint(corners[0][0], corners[0][1], geom.degrees)
corner_NW_deg = geom.SpherePoint(corners[2][0], corners[2][1], geom.degrees)
corner_SE_pix = anchor_wcs.skyToPixel(corner_SE_deg)
corner_NW_pix = anchor_wcs.skyToPixel(corner_NW_deg)

xmin = int(np.floor(corner_SE_pix[0]))
ymin = int(np.floor(corner_SE_pix[1]))
xmax = int(np.floor(corner_NW_pix[0]))
ymax = int(np.floor(corner_NW_pix[1]))

new_img = Image(geom.Box2I(minimum=geom.Point2I(x=xmin, y=ymin),
                           maximum=geom.Point2I(x=xmax, y=ymax)),
                dtype=np.float32)
new_img_bbox = new_img.getBBox()
print(new_img_bbox)

del corner_SE_deg, corner_NW_deg, corner_SE_pix, corner_NW_pix
del xmin, ymin, xmax, ymax
del new_img




Create the arrays of `dataId`s and images to be passed to `GetTemplateTask`.

In [ ]:
# this should NOT be deleted??

all_dataIds = [anchor_dataId]
all_images = [anchor_image]
for i in range(len(add_tracts_and_patches)):
    temp_tract = add_tracts_and_patches[i][0]
    temp_patch = add_tracts_and_patches[i][1]
    temp_dataId = {'tract': temp_tract, 'patch': temp_patch,
                   'band': use_filter}
    temp_image = butler.get('deepCoadd_calexp', dataId=temp_dataId)

    # CCW sanity check that all cdMatrix are the same for each deepCoadd (to delete)
    #x = temp_image.getWcs().getFitsMetadata()
    #print(x['CD1_1'],x['CD1_2'],x['CD2_1'],x['CD2_2'])
    # end CCW sanity check
    
    all_images.append(temp_image)
    all_dataIds.append(temp_dataId)
    #del temp_tract, temp_patch, temp_dataId, temp_image

Option to print the `dataId`s.

In [ ]:
for dataId in all_dataIds:
    print(dataId)

Define `getTemplateTask`. 

No configuration changes are needed.

In [ ]:
getTemplateTask = GetTemplateTask()

Create the custom cutout as `newimage`.

>**Notice:** Some input patch images might be skipped for not, in the end, overlapping with the defined bounding box after all. This was expected, as described in Section 2.3. 

In [ ]:
newimage = getTemplateTask.run(all_images, new_img_bbox,
                               anchor_wcs, all_dataIds)

## 4. Visualize the custom cutout

Extract metadata for the custom cutout.

In [ ]:
newimage_wcs = newimage.template.getWcs()
newimage_wcs_fmd = newimage.template.getWcs().getFitsMetadata()
newimage_bbox = newimage.template.getBBox()
newimage_extent = (newimage_bbox.beginX, newimage_bbox.endX,
                   newimage_bbox.beginY, newimage_bbox.endY)

### 4.1. Display with matplotlib

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.subplot(projection=WCS(newimage_wcs_fmd))
plt.imshow(newimage.template.image.array, vmin=0.01, vmax=0.5,
           extent=newimage_extent, origin='lower', cmap='gray')
plt.show()

### 4.2. Display with `afwDisplay`

In [ ]:
fig = plt.figure(figsize=(8, 8))
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(newimage.template.image)
plt.show()

### 4.3. Display with Firefly

Firefly allows for interactive image display and manipulation.
See DP0.2 tutorial notebook 03b for a full demonstration of how to use Firefly within the Notebook Aspect.

Reset the `afwDisplay` backend to be Firefly and start the display.
A new tab will open, containing the Firefly interface.

In [ ]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.Display(frame=1)

Display the image in Firely.

In [ ]:
afw_display.mtv(newimage.template)

The Firefly default is to visualize the mask plane with colors. 
Set the mask plane transparency to 100, fully transparent, to see only the pixel data.

In [ ]:
afw_display.setMaskTransparency(100)

### 4.4. Option to save as a FITS file

Save as a FITS file in the home directory.

> **Warning:** Images can take up a lot of disk space. Save and download with caution.

In [ ]:
username = os.environ.get('USER')
fnm = '/home/' + username + '/my_big_cutout.fits'
newimage.template.writeFits(fnm)
#del username, fnm

## 5. Test out large cutouts using custom WCS creation 

Here, use the makeSkyWcs, which constructs a simple FITS SkyWcs with no distortion. The plan: define a new WCS where the CRVAL are the coordinates of object of interest, the cdMatrix is the one defined for all deepCoadds (pixel scale), the CRPIX are defined as the center of the bounding box, and the bounding box is defined by the user as the

From Gregory for testing purposes:

I don't have the answer to that particular question.  I'm sure it is easy but I wouldn't be confident of giving you the minimal recipe.  I would ask on #dm-science-pipelines.  It is entirely possible that there's already a packaged-up solution to this.

There are two sub-use cases:

The desired cutout crosses patch boundaries but is within a single tract.  In that case the pixel grids are exactly compatible, so no reprojection is needed.  You just have to pull the pixels from the relevant patches (typically 2 but potentially up to no more than 4, unless the desired cutout is larger than the patch scale itself) and drop them into a single array.  If you're not reprojecting, at the end you have to be careful to preserve the CRPIX/CRVAL values of the inputs and not attempt to recenter the WCS on the cutout, because it's still on a tangent plane relative to the tract center.

The cutout crosses tract boundaries.  In that case the best solution, I imagine, is to reproject all the inputs (again, typically 2-4 for moderately-sized cutouts) to a new tangent plane centered at the target and with north up.

In [ ]:
# here just see what the various WCS formats are, so I know how to construct new one
#print('WCS format from Melissas newimage:', newimage.template.getWcs())

# getFitsMetadata returns a PropertyList which can be accessed as:
#
#print(x.names())
#print(x)
#x['CRPIX2']



Below, define the parameters of the new WCS for the cutout. This will be input to makeSkyWcs as `newWCS` in this cell.

In [ ]:

# input formats needed for this function
# makeSkyWcs(crpix: lsst.geom.Point2D, crval: lsst.geom.SpherePoint, 
#            cdMatrix: numpy.ndarray[numpy.float64[2, 2]], 
#            projection: str = ‘TAN’) -> lsst.afw.geom.SkyWcs

# here you want the center (tangent point) to be the object of interest:
crval = geom.SpherePoint(my_ra, my_dec, geom.degrees) 

# Now center pixel should be the center w.r.t. the bounding box
# use melissa's bounding box for consistency:
print('new img bbox', new_img_bbox)
crpix1 = (new_img_bbox.x.max-new_img_bbox.x.min)/2
crpix2 = (new_img_bbox.y.max-new_img_bbox.y.min)/2
crpix = geom.Point2D(crpix1, crpix2)

# I think cdMatrix is the same for all deepCoadds so just grab one from Section 3
# TBD: make sure the order is correct, as expected in makeSkyWcs
# x = newimage.template.getWcs().getFitsMetadata()
x = temp_image.getWcs().getFitsMetadata()
# x can use the anchor image because 
# the pixel scale is the same everywehre
cdMatrix = [x['CD1_1'],x['CD1_2']],[x['CD2_1'],x['CD2_2']]
projection = 'TAN'

# Make the new WCS:
newWCS = makeSkyWcs(crpix, crval, cdMatrix, projection)
print('Newly generated WCS for the cross-tract cutout:', newWCS)
print('Compare to the anchor cutout:',anchor_wcs)


# update Melissa's bounding box to the pixel values of the newWCS:
corner_SE_deg = geom.SpherePoint(corners[0][0], corners[0][1], geom.degrees)
corner_NW_deg = geom.SpherePoint(corners[2][0], corners[2][1], geom.degrees)
corner_SE_pix = newWCS.skyToPixel(corner_SE_deg)
corner_NW_pix = newWCS.skyToPixel(corner_NW_deg)

xmin = int(np.floor(corner_SE_pix[0]))
ymin = int(np.floor(corner_SE_pix[1]))
xmax = int(np.floor(corner_NW_pix[0]))
ymax = int(np.floor(corner_NW_pix[1]))

new_img2 = Image(geom.Box2I(minimum=geom.Point2I(x=xmin, y=ymin),
                           maximum=geom.Point2I(x=xmax, y=ymax)),
                dtype=np.float32)
new_img2_bbox = new_img2.getBBox()

# Generate a new cross-tract cutout with the new WCS:
newimage2 = getTemplateTask.run(all_images, new_img2_bbox,
                               newWCS, all_dataIds)


newimage2_bbox = newimage2.template.getBBox()
newimage2_extent = (newimage2_bbox.beginX, newimage2_bbox.endX,
                   newimage2_bbox.beginY, newimage2_bbox.endY)
newimage2_wcs_fmd = newimage2.template.getWcs().getFitsMetadata()

fig = plt.figure(figsize=(8, 8))
plt.subplot(projection=WCS(newimage2_wcs_fmd))
plt.imshow(newimage2.template.image.array, vmin=0.01, vmax=0.5,
           extent=newimage2_extent, origin='lower', cmap='gray')
plt.show()


For now, use the same bounding box as in Section 3 for easy comparison. So below, re-generate the location of the bounding box corners using `newWCS`

Re-define the meta-data properties for plotting, and then plot thew new image with user-defined WCS

In [ ]:
#fnm = '/home/' + username + '/my_big_cutout_makeskywcs.fits'
#newimage2.template.writeFits(fnm)
#del username, fnm

NOTES:

- check that ra/dec of sources on outskirts are still at the right location

- get the PSF. it comes with the image i got and ask it to provide the PSF for the x,y coordinates you're interested in. you might see some discontinuity in the PSF size FWHM.

- ask it: gimme PSF at this point and the FWHM comes from butler. Andres has a tutorial notebook, just first one is fine. display image you put together and draw on the boundaries of which tract patch (and label and draw lines on it)

- release notebook and call this done after that step is completed. 
try aaron for this review, if not, then jeff

- PSF is stretch goal,  